In [ ]:
!pip install transformers torch matplotlib


In [ ]:
pip install inseq

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model_name = "gpt2"
model_gpt2 = GPT2Model.from_pretrained(model_name, output_attentions=True)
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
reference_summary = """
Ontario has announced a partial reform of its elementary and secondary school curricula to include mandatory learning on coding, in line with Nova Scotia and British Columbia's decisions. This move aligns with other Canadian provinces, such as Estonia, the United Kingdom, and South Korea.
"""

text = """
Ontario recently announced a partial reform of its elementary and secondary school curricula to include mandatory learning on coding, as of September 2022.
"""

In [ ]:
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

# Define the long text
long_text = text
# Split the long text into sentences
sentences = sent_tokenize(long_text)

In [ ]:
encoded_sentences = [tokenizer_gpt2(sentence, return_tensors="pt", padding=True, truncation=True) for sentence in sentences]

In [ ]:
embeddings = []
for encoded_sentence in encoded_sentences:
    with torch.no_grad():
        outputs = model_gpt2(**encoded_sentence)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

In [ ]:
similarity_matrix = cosine_similarity(embeddings)

# Rank sentences based on similarity to other sentences
sentence_scores = similarity_matrix.sum(axis=1)

# Sort sentences by score in descending order
sorted_sentences = [sentences[i] for i in np.argsort(sentence_scores)[::-1]]

# Select the top N sentences for the summary
summary_length = 1  # Adjust as needed
summary = " ".join(sorted_sentences[:summary_length])

# Print the summary
print(summary)



Ontario recently announced a partial reform of its elementary and secondary school curricula to include mandatory learning on coding, as of September 2022.


In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

# Generated summary
generated_summary = summary

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores
scores = scorer.score(generated_summary, reference_summary)


print("----------Rauge-1 Score----------")
print("Pricision : ",scores['rouge1'].precision)
print("Recall : ",scores['rouge1'].recall)
print("Fmeasure : ",scores['rouge1'].fmeasure)


print("\n----------Rauge-L Score----------")
print("Pricision : ",scores['rougeL'].precision)
print("Recall : ",scores['rougeL'].recall)
print("Fmeasure : ",scores['rougeL'].fmeasure)


----------Rauge-1 Score----------
Pricision :  0.4222222222222222
Recall :  0.8260869565217391
Fmeasure :  0.5588235294117647

----------Rauge-L Score----------
Pricision :  0.4222222222222222
Recall :  0.8260869565217391
Fmeasure :  0.5588235294117647


In [ ]:
import inseq

attribution_model = inseq.load_model("Helsinki-NLP/opus-mt-en-it", "saliency")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
out = attribution_model.attribute(
    input_texts=text,

    attributed_fn="contrast_prob_diff",
    # Special argument to specify the contrastive target, used by the contrast_prob_diff function
    contrast_targets=summary,
    attribute_target=True,
    # We also visualize the score used as target using the same function as step score
    step_scores=["contrast_prob_diff"]
)

out.show()

Filling missing position with right-aligned 1:1 position alignments.
Attributing with saliency...:   3%|▎         | 1/35 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/inseq/utils/contrast_utils.py:148: UserWarning: Contrastive inputs do not match original inputs when using a contrastive attributed function.
By default we force the original inputs to be used (i.e. only the contrastive predicted target is different).
This is a requirement for gradient-based attribution method, as contrastive inputs don't participate in gradient computation.
For attribution methods with less stringent requirements, set --contrast_force_inputs to True to use the contrastive inputs for attribution instead.
  warnings.warn(
Attributing with saliency...: 100%|██████████| 35/35 [00:33<00:00,  1.03it/s]


,▁i → ▁L,ts → ',▁e → O,le → n,menta → tario,ry → ▁ha,▁and → ▁recentemente,▁seconda → ▁annunciato,ry → ▁una,▁s → ▁parziale,cho → ▁riforma,ol → ▁dei,▁curricul → ▁suoi,a → ▁programmi,▁to → ▁scolastici,▁include → ▁elementari,▁mandato → ▁e,ry → ▁secondari,▁le → ▁per,ar → ▁includere,ning → ▁l,▁on → ',▁co → apprendimento,ding → ▁obbligatorio,", → ▁sulla",▁as → ▁codifica,"▁of → ,",▁Se → ▁a,p → ▁partire,te → ▁dal,mber → ▁settembre,▁2022,.,</s>
▁On,0.181,0.077,0.234,0.145,0.084,0.02,0.015,0.016,0.01,0.013,0.009,0.007,0.011,0.007,0.004,0.004,0.011,0.006,0.012,0.011,0.007,0.038,0.008,0.005,0.007,0.009,0.011,0.01,0.008,0.012,0.015,0.0,0.0,0.0
t,0.118,0.052,0.111,0.122,0.093,0.013,0.011,0.012,0.006,0.008,0.006,0.004,0.005,0.004,0.003,0.003,0.006,0.004,0.006,0.006,0.004,0.023,0.005,0.003,0.004,0.005,0.006,0.006,0.004,0.006,0.009,0.0,0.0,0.0
ario,0.19,0.086,0.166,0.238,0.289,0.032,0.029,0.026,0.014,0.021,0.012,0.008,0.014,0.012,0.006,0.006,0.011,0.013,0.016,0.014,0.01,0.042,0.015,0.007,0.009,0.01,0.016,0.011,0.007,0.013,0.017,0.0,0.0,0.0
▁recently,0.054,0.041,0.047,0.055,0.051,0.088,0.143,0.044,0.024,0.025,0.015,0.012,0.015,0.013,0.006,0.007,0.019,0.012,0.026,0.02,0.012,0.032,0.016,0.01,0.013,0.016,0.025,0.025,0.023,0.019,0.022,0.0,0.0,0.0
▁announced,0.069,0.061,0.07,0.048,0.046,0.142,0.296,0.289,0.065,0.042,0.04,0.023,0.019,0.017,0.008,0.01,0.026,0.014,0.056,0.033,0.026,0.039,0.027,0.013,0.024,0.024,0.034,0.026,0.021,0.021,0.025,0.0,0.0,0.0
▁a,0.013,0.008,0.01,0.01,0.009,0.012,0.023,0.022,0.026,0.014,0.012,0.005,0.004,0.004,0.003,0.004,0.005,0.004,0.007,0.005,0.004,0.006,0.004,0.003,0.003,0.005,0.006,0.005,0.006,0.004,0.005,0.0,0.0,0.0
▁partial,0.019,0.019,0.015,0.013,0.015,0.026,0.049,0.052,0.078,0.258,0.073,0.02,0.016,0.011,0.009,0.011,0.022,0.009,0.021,0.019,0.013,0.015,0.014,0.015,0.011,0.012,0.016,0.013,0.012,0.009,0.011,0.0,0.0,0.0
▁reform,0.028,0.03,0.024,0.017,0.014,0.038,0.06,0.081,0.085,0.145,0.318,0.031,0.023,0.019,0.011,0.014,0.018,0.015,0.034,0.032,0.017,0.02,0.018,0.013,0.019,0.026,0.024,0.014,0.012,0.017,0.023,0.0,0.0,0.0
▁of,0.009,0.01,0.008,0.009,0.009,0.011,0.016,0.016,0.02,0.027,0.041,0.015,0.013,0.007,0.006,0.007,0.009,0.005,0.01,0.008,0.005,0.007,0.005,0.004,0.005,0.007,0.008,0.006,0.005,0.006,0.008,0.0,0.0,0.0
▁its,0.014,0.013,0.015,0.015,0.017,0.019,0.013,0.015,0.019,0.019,0.02,0.021,0.043,0.009,0.009,0.018,0.015,0.008,0.013,0.013,0.007,0.009,0.008,0.007,0.007,0.008,0.012,0.008,0.008,0.007,0.009,0.0,0.0,0.0
